In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim

mapping = {'verified':0,'reviewTime':1,'reviewerID':2,'asin':3,"reviewText":4,"summary":5,"unixReviewTime":6,"sentiment":7,"id":8}
ls = []

for line in open('Data/music_reviews_train.json'):
    review_data = json.loads(line)
    tmp = [None]*len(mapping)
    for key in review_data:
        if key in mapping:
            if key == "sentiment":
                
                tmp[mapping[key]] = 1 if review_data[key] == "positive" else 0
            else:
                tmp[mapping[key]] = review_data[key]
    ls.append(tmp)
#df = pd.read_json('Data/music_reviews_train.json', typ='series', orient='records')

In [ ]:
ls_dev = []
for line in open('Data/music_reviews_dev.json'):
    review_data_dev = json.loads(line)
    tmp = [None]*len(mapping)
    for key in review_data_dev:
        if key in mapping:
            if key == "sentiment":
                
                tmp[mapping[key]] = 1 if review_data_dev[key] == "positive" else 0
            else:
                tmp[mapping[key]] = review_data_dev[key]
    ls_dev.append(tmp)

In [ ]:
ls_test = []
for line in open('Data/music_reviews_test_masked.json'):
    print(line)
    break
    review_data_dev = json.loads(line)
    tmp = [None]*len(mapping)
    for key in review_data_dev:
        if key in mapping:
            tmp[mapping[key]] = review_data_dev[key]
    ls_test.append(tmp)

In [ ]:
df = pd.DataFrame(ls)
df = df.drop(df[df[4].isna()].index) #Drops all None values for reviewtext

#DEV
df_dev = pd.DataFrame(ls_dev)
df_dev = df_dev.drop(df_dev[df_dev[4].isna()].index) #Drops all None values for reviewtext

#TEST
df_test = pd.DataFrame(ls_test)
df_test = df_test.drop(df_test[df_test[4].isna()].index) #Drops all None values for reviewtext

In [ ]:
mp = {value:key for key, value in mapping.items()}
df = df.rename(columns=mp)
target = np.array(df["sentiment"])

#DEV
df_dev = df_dev.rename(columns=mp)
target_dev = np.array(df_dev["sentiment"])

#TEST
df_test = df_test.rename(columns=mp)
target_test = np.array(df_test["sentiment"])

In [ ]:
from collections import Counter

def oneHot(sentence):
    output = [0]*len(vocab)
    for word in sentence:
        if word in vocab:
            output[words2idx[word]] = 1
    return output

freqs = defaultdict(int)

for line in df["reviewText"]:
    for word in line.split():
        freqs[word] +=1

words = Counter(freqs)
w = words.most_common(1000)
vocab = set([x[0] for x in w])
words2idx = {x:idx for idx,x in enumerate(vocab)}
X = []

for line in df["reviewText"]:
    X.append(oneHot(line.split()))

In [ ]:
# DEV
X_dev = []
for line in df_dev["reviewText"]:
    X_dev.append(oneHot(line.split()))

# TEST
X_test = []
for line in df_test["reviewText"]:
    X_test.append(oneHot(line.split()))

In [ ]:
from sklearn.svm import SVC
clf = SVC(verbose=True,max_iter=100)
clf.fit(X, target)

In [ ]:
clf.score(X,target)

In [ ]:
clf.score(X_dev,target_dev)

In [ ]:
sum(df["sentiment"])/len(df["sentiment"])

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')
lr.fit(X, target)
#lr.score(X, target)

In [ ]:
lr.score(X,target)

In [ ]:
lr.score(X_dev,target_dev)

In [ ]:
lr.predict(X_test)